In [1]:
# !pip install numpy num2words nltk pandas Observations
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

In [3]:
def convert_numbers(k):
    for i in range(len(k)):
        try:
            num2words(int(k[i]))
            k[i] = " "
        except:
            pass
    return k

def get_cosine(a, b):
    return 1 - spatial.distance.cosine(a, b)

def preprocess(pd):
    pd = pd.str.lower()
    pd = pd.str.replace('[{}]'.format('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n\t'), ' ')
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
    pd = pd.apply(lambda x: convert_numbers(x))
    pd = pd.str.join(' ')
    pd = pd.str.replace('[{}]'.format(string.punctuation), ' ')
    
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])    
    pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
    pd = pd.apply(lambda x: [stemmer.stem(y) for y in x])
    return pd

def processReviews(reviews, window=5, MAX_VOCAB_SIZE=1000):
#     vectorizer = SkipGramVectorizer(analyzer="word",stop_words="english",
#                                          max_features=MAX_VOCAB_SIZE,max_df=.75,min_df=10, k = window,ngram_range=(1,2))
    vectorizer = CountVectorizer(analyzer="word",tokenizer=None)
#                                       , preprocessor=None,stop_words="english",max_features=MAX_VOCAB_SIZE,max_df=.5,min_df=5)
    train_data_features = vectorizer.fit_transform(reviews)
    words = vectorizer.get_feature_names()
    vocabulary = dict(zip(words,np.arange(len(words))))
    inv_vocabulary = dict(zip(np.arange(len(words)),words))
    wordOccurenceMatrix = train_data_features.toarray()
    return wordOccurenceMatrix, vocabulary

In [4]:
embeddings_index = {}
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [6]:
dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", header=None)
dataset = dataset.head(1000)

In [7]:
dataset[6] = preprocess(dataset[5])

In [8]:
dataset[7] = dataset[6].apply(lambda x: " ".join(x))

In [9]:
matrix, vocabulary = processReviews(dataset[7].values)

In [10]:
docs_edges = []
for idx, doc in enumerate(dataset[6].values):
    edges = []
    for i in doc:
        for j in doc:
            if i != j:
                try:
                    a = embeddings_index[i]
                    b = embeddings_index[j]
                    if get_cosine(a, b) > 0.5:
                        edges.append((vocabulary[i], vocabulary[j]))
                except:
                    pass
    docs_edges.append(edges)
    
    if idx % 300 == 0:
        print(idx)

0
300
600
900


In [11]:
edge_dict = {}
for i in docs_edges:
    for j in i:
#         print(j)
        try:
            edge_dict[j[0]] += [j[1]]
        except:
            edge_dict[j[0]] = [j[1]]

In [12]:
import imp

In [13]:
import lda2

In [14]:
lambda_param = 0.8
N_TOPICS = 10

In [17]:
imp.reload(lda2)
sampler = lda2.LdaSampler(n_topics=N_TOPICS, lambda_param=lambda_param)

In [18]:
for it, phi in enumerate(sampler.run(matrix, edge_dict)):
    print "Iteration", it
    print "Likelihood", sampler.loglikelihood(docs_edges)

Iteration 0
Likelihood -71583.11218166549
Iteration 1
Likelihood -68842.44884274623
Iteration 2
Likelihood -67635.13516758756
Iteration 3
Likelihood -66902.40764089416
Iteration 4
Likelihood -66252.16934392933
Iteration 5
Likelihood -65966.3182530556
Iteration 6
Likelihood -65736.0316593655
Iteration 7
Likelihood -65455.64990137747
Iteration 8
Likelihood -65305.02106596165
Iteration 9
Likelihood -64996.909193830616
Iteration 10
Likelihood -64955.81816349574
Iteration 11
Likelihood -64972.66005038114
Iteration 12
Likelihood -64848.04985113517
Iteration 13
Likelihood -64728.383807446204
Iteration 14
Likelihood -64857.22927147303
Iteration 15
Likelihood -64763.03982860091
Iteration 16
Likelihood -64538.23236013443
Iteration 17
Likelihood -64562.26481234656
Iteration 18
Likelihood -64376.84546127207
Iteration 19
Likelihood -64417.42715586826
Iteration 20
Likelihood -64265.471159214285
Iteration 21
Likelihood -64224.64299657952
Iteration 22
Likelihood -64290.77760295536
Iteration 23
Likelih